# 🦜🔗 LangChain agent with human tool

In [ ]:
#%pip install -r requirements.txt

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.tools import ShellTool
from langgraph.prebuilt.tool_executor import ToolExecutor
from helpers import llm

cold_llm = llm.with_config({"temperature" : 0})
tools = [ShellTool()]
prompt = hub.pull("reactagent/openai-functions-agent")
agent_runnable = create_openai_functions_agent(cold_llm, tools, prompt)
tool_executor = ToolExecutor(tools)

In [ ]:
from typing import TypedDict, Annotated, Union
from langchain_core.agents import AgentAction, AgentFinish
import operator

class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [ ]:
from langchain_core.agents import AgentFinish
from langchain_core.agents import AgentActionMessageLog

def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    agent_action :AgentActionMessageLog = data["agent_outcome"]
    if agent_action.tool == 'terminal':
        print(agent_action)
        response = input(prompt=f"[y/n] continue with shell execution: {agent_action.tool_input}?")
        if response == "y":
            output = tool_executor.invoke(agent_action)
        else:
            output = "This specific terminal command not permitted by user. Try a different terminal command or return unfinished."
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

app = workflow.compile()

In [ ]:
from helpers import graph_agent_output_printer
inputs={"input": "Count the lines of all python notebooks in the current directory."}
output = app.invoke(inputs)
graph_agent_output_printer(output)


In [ ]:
inputs={"input": "Send a mail with the subject 'donald trump hat doofe ohren' to donald@trump.com."}
output = app.invoke(inputs)
graph_agent_output_printer(output)